In [1]:
!pip install -q --upgrade ultralytics wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.1 MB/s eta 0:00:00


In [2]:
import os, random, numpy as np, torch, pathlib, pandas as pd
from ultralytics import YOLO
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/drive


### Fixar a *seed* para reprodutibilidade

Treinos de redes neuronais envolvem diversas fontes de aleatoriedade (inicialização de pesos, ordem dos mini-batches, data-augmentation, etc.).  
Se quisermos comparar vários modelos em condições **idênticas** ou voltar a
regerar resultados mais tarde, temos de travar essa aleatoriedade definindo
uma *seed* única nos principais geradores de números aleatórios.

https://docs.pytorch.org/docs/stable/notes/randomness.html

In [3]:
def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed); os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

### 2.1  Preparação do conjunto de dados no Google Colab

Antes de iniciar o treino é necessário disponibilizar o dataset no
filesystem da instância Colab. Para tal, procede-se a (i) copiar o ficheiro
comprimido a partir do Google Drive, (ii) descomprimir o conteúdo e
(iii) garantir que o ficheiro `data.yaml` se encontra na diretoria de
trabalho.

```bash
# (i) Copiar o ficheiro ZIP do Google Drive para o directório actual
!cp /content/drive/MyDrive/apvc_yolo/chestx_det10_yolo.zip .

# (ii) Descomprimir o ZIP de forma silenciosa (opção -q → “quiet”)
!unzip -q chestx_det10_yolo.zip

# (iii) Colocar o ficheiro data.yaml na raiz do projecto
!cp chestx_det10_yolo/data.yaml .


In [4]:
!cp /content/drive/MyDrive/apvc_yolo/chestx_det10_yolo.zip .
!unzip -q chestx_det10_yolo.zip

In [5]:
!cp chestx_det10_yolo/data.yaml .

In [6]:
#@title ⚙️ Caminhos
DATASET_DIR = '/content/drive/MyDrive/YOLO/data'   # ⬅️ ajuste aqui
DATA_YAML    = f'data.yaml'
PROJECT_DIR  = '/content/drive/MyDrive/YOLO/runs_workflow'

assert os.path.exists(DATA_YAML), 'data.yaml não encontrado'
print('Dataset:', DATA_YAML)
print('Resultados →', PROJECT_DIR)


Dataset: data.yaml
Resultados → /content/drive/MyDrive/YOLO/runs_workflow


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1Baseline – YOLOv8s
Treino rápido (150 épocas) para obter referência.


In [8]:
#@title 🚀 Treinar baseline YOLOv8s
set_seed(42)
y8s_base = YOLO('yolov8s.pt')
y8s_base.train(
    data=DATA_YAML,
    epochs=150,
    imgsz=640,
    batch=64,
    seed=42,
    lr0=5e-4,
    optimizer='AdamW',
    cos_lr=True,
    warmup_epochs=3,
    patience=10,
    amp=True,
    project=PROJECT_DIR,
    name='yolov8s_base',
    verbose=False,
    plots=True,
)
best_s_base = f'{PROJECT_DIR}/yolov8s_base/weights/best.pt'
metrics_s_base = YOLO(best_s_base).val(data=DATA_YAML, split='val', imgsz=640, batch=16, plots=True)


  0%|          | 29.6k/21.5M [00:00<00:00, 37.9MB/s]


Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_base, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrai

100%|██████████| 755k/755k [00:00<00:00, 112MB/s]

Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

Model summary: 129 layers, 11,139,470 parameters, 11,139,454 gradients, 28.7 GFLOPs

Transferred 349/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...


119 epochs completed in 0.527 hours.
Optimizer stripped from /content/drive/MyDrive/YOLO/runs_workflow/yolov8s_base/weights/last.pt, 22.5MB
Optimizer stripped from /content/drive/MyDrive/YOLO/runs_workflow/yolov8s_base/weights/best.pt, 22.5MB

Validating /content/drive/MyDrive/YOLO/runs_workflow/yolov8s_base/weights/best.pt...
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.50it/s]


                   all        542       1476      0.634      0.409      0.458      0.231
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to /content/drive/MyDrive/YOLO/runs_workflow/yolov8s_base
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4136.5±1238.1 MB/s, size: 401.1 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:04<00:00,  8.38it/s]


                   all        542       1476      0.637      0.408      0.458      0.231
           Atelectasis         48         51      0.565      0.471       0.42      0.165
         Calcification         38         67      0.561      0.328      0.367      0.146
         Consolidation        289        446      0.678      0.594      0.615        0.3
              Effusion        252        372      0.597      0.541      0.561      0.241
             Emphysema         39         66      0.927      0.545      0.741       0.58
              Fibrosis         82        120      0.604      0.333      0.399       0.21
              Fracture         76        115       0.62      0.322      0.368      0.134
                  Mass         30         31      0.694      0.419      0.502      0.251
                Nodule         77        166      0.559      0.265       0.32      0.154
          Pneumothorax         35         42      0.562      0.262      0.286      0.132
Speed: 0.5ms preproce

## Baseline – YOLOv8m
Treino do modelo médio para avaliar potencial ganho de precisão.


In [ ]:
#Treinar baseline YOLOv8m
set_seed(42)
y8m_base = YOLO('yolov8m.pt')
y8m_base.train(
    data=DATA_YAML,
    epochs=150,
    imgsz=640,
    batch=64,
    seed=42,
    lr0=5e-4,
    optimizer='AdamW',
    cos_lr=True,
    warmup_epochs=3,
    patience=10,
    amp=True,
    project=PROJECT_DIR,
    name='yolov8m_base',
    verbose=False,
    plots=True,
)
best_m_base = f'{PROJECT_DIR}/yolov8m_base/weights/best.pt'
metrics_m_base = YOLO(best_m_base).val(data=DATA_YAML, split='val', imgsz=640, batch=16, plots=True)


100%|██████████| 49.7M/49.7M [00:00<00:00, 217MB/s]


Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m_base, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretrai

train: Scanning /content/chestx_det10_yolo/labels/train.cache... 3001 images, 681 backgrounds, 0 corrupt: 100%|██████████| 3001/3001 [00:00<?, ?it/s]

train: /content/chestx_det10_yolo/images/train/57467.png: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1858.7±1174.4 MB/s, size: 377.5 KB)


...


89 epochs completed in 0.565 hours.
Optimizer stripped from /content/drive/MyDrive/YOLO/runs_workflow/yolov8m_base/weights/last.pt, 52.0MB
Optimizer stripped from /content/drive/MyDrive/YOLO/runs_workflow/yolov8m_base/weights/best.pt, 52.0MB

Validating /content/drive/MyDrive/YOLO/runs_workflow/yolov8m_base/weights/best.pt...
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.36it/s]


                   all        542       1476      0.561      0.408      0.419      0.216
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to /content/drive/MyDrive/YOLO/runs_workflow/yolov8m_base
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3897.3±909.7 MB/s, size: 401.1 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:04<00:00,  7.65it/s]


                   all        542       1476      0.562      0.407      0.419      0.216
           Atelectasis         48         51       0.38      0.235      0.281      0.104
         Calcification         38         67      0.457      0.226       0.25      0.101
         Consolidation        289        446      0.574      0.614       0.59      0.287
              Effusion        252        372      0.491      0.605      0.547      0.234
             Emphysema         39         66      0.867      0.594      0.722      0.568
              Fibrosis         82        120      0.488      0.442      0.423      0.218
              Fracture         76        115      0.639       0.33      0.402      0.137
                  Mass         30         31      0.582      0.548      0.447      0.251
                Nodule         77        166      0.502      0.289      0.287      0.132
          Pneumothorax         35         42      0.643       0.19      0.241      0.132
Speed: 0.5ms preproce

In [ ]:
# Comparação preliminar v8s vs v8m — versão robusta
import os, pandas as pd
from ultralytics import YOLO

def _v(x):
    """Retorna x() se for função, caso contrário x."""
    return x() if callable(x) else x

def val_metrics(pt_path: str, tag: str) -> dict:
    m = YOLO(pt_path).val(
        data=DATA_YAML,
        split='val',
        imgsz=640,
        batch=16,
        plots=False
    )
    b = m.box  # objeto Metric
    return {
        'modelo':   tag,
        'mAP50-95': _v(b.map),      # mean AP 0.50‒0.95
        'mAP50':    _v(b.map50),    # AP @ 0.50 IoU
        'precisão': _v(b.mp),       # mean precision
        'recall':   _v(b.mr),       # mean recall
    }

records = []
for tag, ckpt in [('YOLOv8s_base', best_s_base),
                  ('YOLOv8m_base', best_m_base)]:
    if ckpt and os.path.exists(ckpt):
        records.append(val_metrics(ckpt, tag))
    else:
        print(f'⚠️  {tag}: checkpoint não encontrado ({ckpt}).')

df_baselines = (
    pd.DataFrame(records)
      .sort_values('mAP50-95', ascending=False)
      .reset_index(drop=True)
)
display(df_baselines)


Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2914.6±1408.8 MB/s, size: 396.5 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:02<00:00, 12.26it/s]


                   all        542       1476      0.637      0.408      0.458      0.231
           Atelectasis         48         51      0.565      0.471       0.42      0.165
         Calcification         38         67      0.561      0.328      0.367      0.146
         Consolidation        289        446      0.678      0.594      0.615        0.3
              Effusion        252        372      0.597      0.541      0.561      0.241
             Emphysema         39         66      0.927      0.545      0.741       0.58
              Fibrosis         82        120      0.604      0.333      0.399       0.21
              Fracture         76        115       0.62      0.322      0.368      0.134
                  Mass         30         31      0.694      0.419      0.502      0.251
                Nodule         77        166      0.559      0.265       0.32      0.154
          Pneumothorax         35         42      0.562      0.262      0.286      0.132
Speed: 0.1ms preproce

val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:03<00:00, 10.34it/s]

                   all        542       1476      0.562      0.407      0.419      0.216
           Atelectasis         48         51       0.38      0.235      0.281      0.104
         Calcification         38         67      0.457      0.226       0.25      0.101
         Consolidation        289        446      0.574      0.614       0.59      0.287
              Effusion        252        372      0.491      0.605      0.547      0.234
             Emphysema         39         66      0.867      0.594      0.722      0.568
              Fibrosis         82        120      0.488      0.442      0.423      0.218
              Fracture         76        115      0.639       0.33      0.402      0.137
                  Mass         30         31      0.582      0.548      0.447      0.251
                Nodule         77        166      0.502      0.289      0.287      0.132
          Pneumothorax         35         42      0.643       0.19      0.241      0.132
Speed: 0.1ms preproce

,modelo,mAP50-95,mAP50,precisão,recall
0,YOLOv8s_base,0.231135,0.457936,0.636764,0.408099
1,YOLOv8m_base,0.216351,0.418951,0.562325,0.407432


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df_baselines)

### 2.6  Interpretação actualizada dos *baselines*

| Modelo             | mAP<sub>50-95</sub> | mAP<sub>50</sub> | Precisão | Recall |
|--------------------|:-------------------:|:----------------:|:--------:|:------:|
| **YOLOv8s (base)** | **0.231** | **0.458** | **0.637** | **0.408** |
| YOLOv8m (base)     | 0.216 | 0.419 | 0.562 | 0.407 |

**Análise comparativa**

* **Desempenho global** – o YOLOv8 s ultrapassa o YOLOv8 m em +0.015
  (≈ +7 %) de *mAP<sub>50-95</sub>* e em +4 p.p. de precisão, mantendo
  ligeira vantagem em recall.  
* **Eficiência computacional** – apesar do melhor resultado, o YOLOv8 s
  possui cerca de metade dos parâmetros do modelo m, exigindo menos VRAM e
  menor tempo de treino/inferência.  
* **Variabilidade experimental** – a inversão de posições face à execução
  anterior demonstra a sensibilidade do treino a variações estocásticas; a
  função `set_seed()` foi reforçada com
  `torch.backends.cudnn.deterministic=True` e
  `benchmark=False` para aumentar a reprodutibilidade.

**Decisão**

> Tendo em conta a combinação de **melhor desempenho** e **menor custo
> computacional**, o **YOLOv8 s** é seleccionado como modelo base para as
> optimizações subsequentes.


## Hyper‑tuning – YOLOv8s
Pequena grelha de 4 combinações (LR × Batch) com Focal Loss.


In [ ]:
# Grid Search — YOLOv8m (LR × Batch)
search_space = {'lr0': [2e-4, 5e-4], 'batch': [64]}   # ajuste batch conforme VRAM
results = []

def _v(x):           # devolve x() se for função; caso contrário x
    return x() if callable(x) else x

for lr in search_space['lr0']:
    for bs in search_space['batch']:
        run = f'y8s_lr{lr}_bs{bs}'
        print(f'▶️  Treinando {run}')
        set_seed(42)
        YOLO('yolov8s.pt').train(
            data=DATA_YAML,
            epochs=50,
            imgsz=640,
            batch=bs,
            lr0=lr,
            optimizer='AdamW',
            cos_lr=True,
            warmup_epochs=3,
            patience=10,
            amp=True,
            project=PROJECT_DIR,
            name=run,
            verbose=False,
            plots=False)

        # avaliação
        best_pt = f'{PROJECT_DIR}/{run}/weights/best.pt'
        m = YOLO(best_pt).val(
            data=DATA_YAML,
            split='val',
            imgsz=640,
            batch=bs,
            plots=False)

        b = m.box  # Metric object
        results.append({
            'run':       run,
            'lr0':       lr,
            'batch':     bs,
            'mAP50-95':  _v(b.map),
            'mAP50':     _v(b.map50),
            'precision': _v(b.mp),
            'recall':    _v(b.mr)
        })

# ordenar por melhor mAP50-95
tune_df = (
    pd.DataFrame(results)
      .sort_values('mAP50-95', ascending=False)
      .reset_index(drop=True)
)
display(tune_df)

# guardar o melhor checkpoint
best_run = tune_df.loc[0, 'run']
best_m_tuned_pt = f'{PROJECT_DIR}/{best_run}/weights/best.pt'
print('Melhor experimento:', best_run)


▶️  Treinando y8s_lr0.0002_bs64
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0002, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=y8s_lr0.0002_bs64, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=10, perspective

train: Scanning /content/chestx_det10_yolo/labels/train.cache... 3001 images, 681 backgrounds, 0 corrupt: 100%|██████████| 3001/3001 [00:00<?, ?it/s]

train: /content/chestx_det10_yolo/images/train/57467.png: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1997.5±1611.3 MB/s, size: 404.5 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]


...

Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3921.5±934.0 MB/s, size: 367.9 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:04<00:00,  2.25it/s]

                   all        542       1476      0.523      0.398      0.407      0.216
           Atelectasis         48         51      0.319      0.314      0.254      0.106
         Calcification         38         67      0.723      0.254      0.318       0.12
         Consolidation        289        446      0.558      0.619      0.591      0.285
              Effusion        252        372      0.538      0.562      0.552      0.238
             Emphysema         39         66      0.837      0.591      0.758      0.595
              Fibrosis         82        120      0.558       0.41      0.423      0.219
              Fracture         76        115      0.503      0.335      0.329      0.118
                  Mass         30         31      0.416      0.387      0.341      0.205
                Nodule         77        166      0.395      0.247      0.257      0.132
          Pneumothorax         35         42      0.378      0.262      0.243      0.141
Speed: 0.1ms preproce

,run,lr0,batch,mAP50-95,mAP50,precision,recall
0,y8s_lr0.0005_bs64,0.0005,64,0.215987,0.406566,0.522535,0.398043
1,y8s_lr0.0002_bs64,0.0002,64,0.204625,0.405304,0.480312,0.407660


Melhor experimento: y8s_lr0.0005_bs64


### 3.1  Análise do *hyper-tuning* (YOLOv8-s, Batch 64)

| Execução | `lr0` | Batch | mAP<sub>50-95</sub> | mAP<sub>50</sub> | Precisão | Recall |
|----------|:-----:|:-----:|:-------------------:|:----------------:|:--------:|:------:|
| **y8s_lr0.0005_bs64** | **5 × 10⁻⁴** | 64 | **0.216** | **0.407** | **0.523** | 0.398 |
| y8s_lr0.0002_bs64     | 2 × 10⁻⁴ | 64 | 0.205 | 0.405 | 0.480 | **0.408** |

**Síntese dos resultados**

* **Desempenho óptimo** – a configuração `lr0 = 5 e-4` obteve a maior mAP<sub>50-95</sub> (+1,1 p.p.) e a maior precisão (≈ +4 p.p.) relativamente à alternativa.  
* **Compromisso precisão-recall** – embora `lr0 = 2 e-4` apresente +1 p.p. de recall, o decréscimo simultâneo de precisão e mAP global não compensa.  
* **Checkpoint seleccionado** – o modelo *y8s_lr0.0005_bs64* será considerado o **novo ponto de partida (“best tuned”)** para as optimizações seguintes.


## Comparação de todos os modelos
Cria uma tabela ordenada por `mAP50‑95`.

In [17]:
# 📊 Tabela comparativa (pastas conhecidas)
import os, pathlib, pandas as pd
from ultralytics import YOLO

PROJECT_DIR = pathlib.Path(PROJECT_DIR)   # já definido
def _v(x): return x() if callable(x) else x  # compatibilidade API

def val_metrics(pt_path, tag):
    m = YOLO(pt_path).val(data=DATA_YAML, split='val',
                          imgsz=640, batch=16, plots=False)
    b = m.box
    return {'model': tag,
            'mAP50-95': _v(b.map),
            'mAP50': _v(b.map50),
            'precision': _v(b.mp),
            'recall': _v(b.mr)}

# --- relação tag  → pasta exacta ---
folders = {
    'YOLOv8s_base' : 'yolov8s_base',
    'YOLOv8m_base' : 'yolov8m_base',
    'y8s_lr0.0005_bs64': 'y8s_lr0.0005_bs64',
    'y8s_lr0.0002_bs64': 'y8s_lr0.0002_bs64',
}

records = []
for tag, folder in folders.items():
    pt = PROJECT_DIR / folder / 'weights' / 'best.pt'
    if pt.exists():
        print(f'✅  {tag}: encontrado')
        records.append(val_metrics(str(pt), tag))
    else:
        print(f'⚠️  {tag}: {pt} não existe')

df_final = (
    pd.DataFrame(records)
      .sort_values('mAP50-95', ascending=False)
      .reset_index(drop=True)
)
display(df_final)


✅  YOLOv8s_base: encontrado
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3671.2±1149.6 MB/s, size: 406.1 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:02<00:00, 11.42it/s]

                   all        542       1476      0.637      0.408      0.458      0.231
           Atelectasis         48         51      0.565      0.471       0.42      0.165
         Calcification         38         67      0.561      0.328      0.367      0.146
         Consolidation        289        446      0.678      0.594      0.615        0.3
              Effusion        252        372      0.597      0.541      0.561      0.241
             Emphysema         39         66      0.927      0.545      0.741       0.58
              Fibrosis         82        120      0.604      0.333      0.399       0.21
              Fracture         76        115       0.62      0.322      0.368      0.134
                  Mass         30         31      0.694      0.419      0.502      0.251
                Nodule         77        166      0.559      0.265       0.32      0.154
          Pneumothorax         35         42      0.562      0.262      0.286      0.132
Speed: 0.1ms preproce

✅  YOLOv8m_base: encontrado
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,845,550 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3708.6±1579.8 MB/s, size: 402.3 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:03<00:00, 10.19it/s]

...

val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:02<00:00, 12.21it/s]


                   all        542       1476      0.523      0.398      0.407      0.216
           Atelectasis         48         51      0.319      0.314      0.254      0.106
         Calcification         38         67      0.724      0.254      0.318       0.12
         Consolidation        289        446      0.558      0.619      0.591      0.285
              Effusion        252        372      0.538      0.562      0.552      0.238
             Emphysema         39         66      0.837      0.591      0.758      0.595
              Fibrosis         82        120      0.558       0.41      0.423      0.219
              Fracture         76        115      0.503      0.335      0.329      0.118
                  Mass         30         31      0.417      0.387      0.341      0.204
                Nodule         77        166      0.395      0.247      0.257      0.132
          Pneumothorax         35         42      0.378      0.262      0.243      0.141
Speed: 0.1ms preproce

val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:02<00:00, 11.78it/s]


                   all        542       1476       0.48      0.408      0.405      0.205
           Atelectasis         48         51      0.276      0.275      0.255      0.122
         Calcification         38         67      0.798      0.236      0.361      0.118
         Consolidation        289        446      0.581      0.637      0.616       0.29
              Effusion        252        372      0.525      0.567      0.505      0.228
             Emphysema         39         66      0.662      0.591       0.72      0.544
              Fibrosis         82        120      0.467      0.383      0.349      0.191
              Fracture         76        115      0.504      0.348      0.325      0.108
                  Mass         30         31      0.245      0.387      0.341      0.186
                Nodule         77        166      0.359      0.319      0.262      0.121
          Pneumothorax         35         42      0.386      0.333      0.319      0.139
Speed: 0.1ms preproce

,model,mAP50-95,mAP50,precision,recall
0,YOLOv8s_base,0.231135,0.457936,0.636764,0.408099
1,YOLOv8m_base,0.216351,0.418951,0.562325,0.407432
2,y8s_lr0.0005_bs64,0.215942,0.406591,0.522657,0.398021
3,y8s_lr0.0002_bs64,0.204631,0.405377,0.480354,0.407712


In [ ]:
# Treino extra rápido – YOLOv8s (+aug +γ)
set_seed(42)
model = YOLO('yolov8s.pt')

model.train(
    data=DATA_YAML,
    epochs=150,
    imgsz=640,
    batch=64,
    seed=42,
    lr0=5e-4,
    optimizer='AdamW',
    cos_lr=True,
    warmup_epochs=3,
    patience=8,
    amp=True,

    # ---- data augmentation ----
    mosaic=0.8,          # mistura 1.5× maior
    copy_paste=0.2,      # 20 % das imagens
    hsv_v=0.4,           # variação de luminância

    project=PROJECT_DIR,
    name='y8s_aug_gamma',
    verbose=False,
    plots=True,
)


Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=0.8, multi_scale=False, name=y8s_aug_gamma2, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=8, perspective=0.0, plots=True, pose=12.0, pretra

train: Scanning /content/chestx_det10_yolo/labels/train.cache... 3001 images, 681 backgrounds, 0 corrupt: 100%|██████████| 3001/3001 [00:00<?, ?it/s]

train: /content/chestx_det10_yolo/images/train/57467.png: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1695.8±1420.4 MB/s, size: 377.5 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]


...

                   all        542       1476      0.339      0.299      0.263      0.131


EarlyStopping: Training stopped early as no improvement observed in last 8 epochs. Best results observed at epoch 20, best model saved as best.pt.
To update EarlyStopping(patience=8) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

28 epochs completed in 0.127 hours.
Optimizer stripped from /content/drive/MyDrive/YOLO/runs_workflow/y8s_aug_gamma2/weights/last.pt, 22.5MB
Optimizer stripped from /content/drive/MyDrive/YOLO/runs_workflow/y8s_aug_gamma2/weights/best.pt, 22.5MB

Validating /content/drive/MyDrive/YOLO/runs_workflow/y8s_aug_gamma2/weights/best.pt...
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:03<00:00,  1.56it/s]


                   all        542       1476      0.372      0.333      0.278      0.142
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to /content/drive/MyDrive/YOLO/runs_workflow/y8s_aug_gamma2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d2d2608e950>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0

In [ ]:
# Avaliação do treino 'y8s_aug_gamma'
best_aug_pt = f"{PROJECT_DIR}/y8s_aug_gamma/weights/best.pt"
assert os.path.exists(best_aug_pt), "Checkpoint best.pt não encontrado — verifique se o treino terminou."

from ultralytics import YOLO
import pandas as pd

def _v(x):           # compatível com Ultralytics ≥ 8.3 (atributo ou método)
    return x() if callable(x) else x

# Avaliar no conjunto de validação
res = YOLO(best_aug_pt).val(
    data=DATA_YAML,
    split='val',
    imgsz=640,
    batch=16,
    plots=True          # grava PR-curve e Confusion Matrix
)

# Extrair métricas principais
b = res.box
metrics_aug = {
    'model'     : 'YOLOv8s + Aug (y8s_aug_gamma)',
    'mAP50-95'  : _v(b.map),
    'mAP50'     : _v(b.map50),
    'precision' : _v(b.mp),
    'recall'    : _v(b.mr)
}

# Mostrar em formato tabular
pd.DataFrame([metrics_aug])


Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3883.3±963.5 MB/s, size: 401.1 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:03<00:00,  8.88it/s]


                   all        542       1476      0.366      0.152      0.148     0.0679
           Atelectasis         48         51      0.329      0.098      0.092     0.0273
         Calcification         38         67      0.149      0.134       0.11     0.0346
         Consolidation        289        446      0.405      0.278      0.321      0.141
              Effusion        252        372      0.296      0.382      0.252     0.0974
             Emphysema         39         66       0.38      0.348      0.331      0.217
              Fibrosis         82        120      0.219      0.142      0.107     0.0595
              Fracture         76        115          0          0     0.0138    0.00344
                  Mass         30         31      0.257     0.0323     0.0477     0.0161
                Nodule         77        166      0.627      0.101      0.171     0.0786
          Pneumothorax         35         42          1          0     0.0309    0.00455
Speed: 0.5ms preproce

,model,mAP50-95,mAP50,precision,recall
0,YOLOv8s + Aug (y8s_aug_gamma),0.067935,0.147641,0.366213,0.151601


### 4.2  Comparação completa (incluindo o treino `y8s_aug_gamma`)

| Rank | Modelo (identificador)                  | mAP<sub>50-95</sub> (pontuação global) | mAP<sub>50</sub> | Precisão (% acertos) | Recall (% casos detec.) |
|:---:|-----------------------------------------|:---------------------------------------:|:----------------:|:--------------------:|:-----------------------:|
| **1** | **YOLOv8-s _base**                    | **0,231** | **0,458** | **63,7 %** | **40,8 %** |
| 2 | YOLOv8-m _base                           | 0,216 | 0,419 | 56,2 % | 40,7 % |
| 3 | YOLOv8-s (LR 5e-4 · batch 64)            | 0,216 | 0,409 | 52,3 % | 39,8 % |
| 4 | YOLOv8-s (LR 2e-4 · batch 64)            | 0,205 | 0,405 | 48,0 % | 40,8 % |
| 5 | **YOLOv8-s + Aug** (`y8s_aug_gamma`)     | **0,068** | **0,148** | **36,6 %** | **15,2 %** |

**Leitura rápida das métricas**

* **mAP<sub>50-95</sub>** – qualidade média das detecções (valor máximo = 1).  
* **mAP<sub>50</sub>** – mesma métrica com tolerância maior (IoU 0,50).  
* **Precisão** – percentagem de alertas que são realmente lesões (evita falsos-positivos).  
* **Recall** – percentagem de lesões reais que o modelo encontra (evita falsos-negativos).

#### Observações

* O *baseline* **YOLOv8-s** continua a liderar em todas as métricas.  
* O treino com augumentação agressiva (**`y8s_aug_gamma`**) fez a pontuação global cair de 0,231 → 0,068 (-16 p.p.) e reduziu recall para 15 %.  
  * Mosaic muito elevado (0,8) + Copy-Paste sobre radiografias introduziu ruído visual.  
  * Batch 64 + LR alto dificultou a convergência.


> Mantemos **YOLOv8-s (base)** como modelo de referência.  
> Treinaremos **apenas mais uma versão** com augumentação **moderada (Mosaic 0,5, sem Copy-Paste, batch 32, LR 3e-4)**.  



In [ ]:
# Treino finalíssimo – YOLOv8-s (Mosaic suave, batch 32)
set_seed(42)
YOLO('yolov8s.pt').train(
    data=DATA_YAML,
    epochs=60,            # menos épocas → termina em ~10-12 min
    imgsz=640,
    batch=32,
    lr0=3e-4,             # intermédio entre 2e-4 e 5e-4
    optimizer='AdamW',
    cos_lr=True,
    warmup_epochs=3,
    patience=6,           # pára cedo se não melhorar
    amp=True,

    mosaic=0.5,           # 50 % das imagens com Mosaic
    hsv_v=0.3,            # variação de brilho moderada
    copy_paste=0.0,       # desligado

    project=PROJECT_DIR,
    name='y8s_mosaic05_b32',
    verbose=False,
    plots=True,
)

Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.3, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0003, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=0.5, multi_scale=False, name=y8s_mosaic05_b32, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=6, perspective=0.0, plots=True, pose=12.0, pretr

train: Scanning /content/chestx_det10_yolo/labels/train.cache... 3001 images, 681 backgrounds, 0 corrupt: 100%|██████████| 3001/3001 [00:00<?, ?it/s]

train: /content/chestx_det10_yolo/images/train/57467.png: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 709.9±15.5 MB/s, size: 404.5 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:06<00:00,  1.41it/s]


                   all        542       1476      0.561      0.386      0.405      0.209
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to /content/drive/MyDrive/YOLO/runs_workflow/y8s_mosaic05_b32
Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3786.5±1049.5 MB/s, size: 367.9 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:03<00:00,  8.66it/s]


                   all        542       1476      0.558       0.39      0.406      0.209
           Atelectasis         48         51      0.438      0.294      0.303      0.107
         Calcification         38         67      0.695      0.343      0.388      0.147
         Consolidation        289        446       0.59      0.576      0.585      0.278
              Effusion        252        372      0.538      0.556       0.49       0.21
             Emphysema         39         66       0.86      0.651      0.767      0.601
              Fibrosis         82        120      0.545      0.367      0.391      0.199
              Fracture         76        115      0.682       0.33      0.349       0.13
                  Mass         30         31      0.404      0.323      0.303      0.163
                Nodule         77        166      0.459      0.241      0.261      0.137
          Pneumothorax         35         42      0.364      0.214      0.221      0.122
Speed: 0.5ms preproce

TypeError: 'numpy.float64' object is not callable

In [24]:
# Avaliar
best_final = f"{PROJECT_DIR}/y8s_mosaic05_b32/weights/best.pt"

# avaliação
metrics_final = YOLO(best_final).val(
    data=DATA_YAML,
    split='val',
    imgsz=640,
    batch=16,
    plots=True
)

print("mAP50-95 final:", _v(metrics_final.box.map))
print("mAP50 final   :", _v(metrics_final.box.map50))
print("Precisão média:", _v(metrics_final.box.mp))
print("Recall médio  :", _v(metrics_final.box.mr))


Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3243.2±1848.8 MB/s, size: 404.7 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:04<00:00,  8.31it/s]


                   all        542       1476      0.558       0.39      0.406      0.209
           Atelectasis         48         51      0.438      0.294      0.303      0.107
         Calcification         38         67      0.695      0.343      0.388      0.147
         Consolidation        289        446       0.59      0.576      0.585      0.278
              Effusion        252        372      0.538      0.556       0.49       0.21
             Emphysema         39         66       0.86      0.651      0.767      0.601
              Fibrosis         82        120      0.545      0.367      0.391      0.199
              Fracture         76        115      0.682       0.33      0.349       0.13
                  Mass         30         31      0.404      0.323      0.303      0.163
                Nodule         77        166      0.459      0.241      0.261      0.137
          Pneumothorax         35         42      0.364      0.214      0.221      0.122
Speed: 0.6ms preproce

### 4.3  Avaliação do re-treino suave (“YOLOv8-s, Mosaic 0.5, batch 32”)

| Ordem | Modelo                            | mAP<sub>50-95</sub> (pontuação global) | mAP<sub>50</sub> | Precisão | Recall |
|:---:|-----------------------------------|:---------------------------------------:|:----------------:|:--------:|:------:|
| **1** | **YOLOv8-s (base)**                    | **0 .231** | **0 .458** | **63,7 %** | **40,8 %** |
| 2 | YOLOv8-m (base)                       | 0 .216 | 0 .419 | 56,2 % | 40,7 % |
| 3 | YOLOv8-s (LR 5e-4 · batch 64)         | 0 .216 | 0 .409 | 52,3 % | 39,8 % |
| 4 | YOLOv8-s (LR 2e-4 · batch 64)         | 0 .205 | 0 .405 | 48,0 % | 40,8 % |
| 5 | YOLOv8-s + Aug (Mosaic 0.8 / CP 0.2) | 0 .068 | 0 .148 | 36,6 % | 15,2 % |
| **6** | **YOLOv8-s, Mosaic 0.5, batch 32**    | **0 .209** | **0 .406** | **55,8 %** | **38,9 %** |

#### Leitura rápida

* O novo re-treino (linha 6) melhorou **claramente** face ao experimento agressivo da linha 5, mas **continua abaixo do baseline** em todas as métricas (+/- 2 p.p.).  
* A precisão (55,8 %) recuperou, mas ainda está 8 p.p. atrás da versão original.  
* O recall (38,9 %) aproxima-se do baseline (40,8 %), mas não o supera.

#### Conclusão final

> Mesmo com augmentação mais suave e batch reduzido, **o re-treino não consegue ultrapassar o YOLOv8-s (base)**.  
> Mantemos portanto o **checkpoint original “YOLOv8-s (base)”**


In [ ]:
# Fine-tune relâmpago – 15 épocas, LR 1e-4, batch 32
set_seed(42)
model = YOLO(f"{PROJECT_DIR}/yolov8s_base/weights/best.pt")  # checkpoint atual

model.train(
    data=DATA_YAML,
    epochs=50,
    imgsz=640,
    batch=32,
    lr0=1e-4,          # passo pequeno para refinamento
    optimizer='AdamW',
    cos_lr=True,
    warmup_epochs=1,
    patience=6,        # pára cedo se não melhorar
    amp=True,
    mosaic=0.0,        # nenhuma aug adicional
    copy_paste=0.0,
    hsv_v=0.2,
    project=PROJECT_DIR,
    name='y8s_finetune15',
    verbose=False,
    plots=False,
)


Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.2, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/YOLO/runs_workflow/yolov8s_base/weights/best.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=y8s_finetune152, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True

train: Scanning /content/chestx_det10_yolo/labels/train.cache... 3001 images, 681 backgrounds, 0 corrupt: 100%|██████████| 3001/3001 [00:00<?, ?it/s]

train: /content/chestx_det10_yolo/images/train/57467.png: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1381.9±1330.5 MB/s, size: 404.5 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  4.67it/s]

                   all        542       1476      0.571      0.445      0.443      0.225
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 0.9ms postprocess per image


Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3007.6±343.7 MB/s, size: 367.9 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:03<00:00,  8.94it/s]


                   all        542       1476      0.562      0.446      0.442      0.224
           Atelectasis         48         51      0.463       0.49      0.381      0.136
         Calcification         38         67      0.556      0.358      0.368      0.135
         Consolidation        289        446      0.646      0.599      0.601      0.289
              Effusion        252        372      0.565      0.581      0.548      0.229
             Emphysema         39         66      0.844      0.573      0.716      0.573
              Fibrosis         82        120      0.545      0.467      0.434      0.227
              Fracture         76        115       0.57      0.381      0.379      0.131
                  Mass         30         31      0.544      0.452      0.436      0.245
                Nodule         77        166       0.45      0.325      0.304       0.15
          Pneumothorax         35         42      0.432      0.238      0.251      0.124
Speed: 0.5ms preproce

In [27]:
best_ft = f"{PROJECT_DIR}/y8s_finetune15/weights/best.pt"
res = YOLO(best_ft).val(data=DATA_YAML, split='val', imgsz=640, batch=16)

print("Novo mAP50-95:", _v(res.box.map))
print("Novo mAP50   :", _v(res.box.map50))
print("Precisão     :", _v(res.box.mp))
print("Recall       :", _v(res.box.mr))

Ultralytics 8.3.162 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3663.1±1458.5 MB/s, size: 404.7 KB)


val: Scanning /content/chestx_det10_yolo/labels/val.cache... 542 images, 83 backgrounds, 0 corrupt: 100%|██████████| 542/542 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:03<00:00,  8.54it/s]


                   all        542       1476      0.562      0.446      0.442      0.224
           Atelectasis         48         51      0.463       0.49      0.381      0.136
         Calcification         38         67      0.556      0.358      0.368      0.135
         Consolidation        289        446      0.646      0.599      0.601      0.289
              Effusion        252        372      0.565      0.581      0.548      0.229
             Emphysema         39         66      0.844      0.573      0.716      0.573
              Fibrosis         82        120      0.545      0.467      0.434      0.227
              Fracture         76        115       0.57      0.381      0.379      0.131
                  Mass         30         31      0.544      0.452      0.436      0.245
                Nodule         77        166       0.45      0.325      0.304       0.15
          Pneumothorax         35         42      0.432      0.238      0.251      0.124
Speed: 0.4ms preproce

### 5. Escolha final do modelo – ênfase na sensibilidade clínica

| Modelo                               | mAP<sub>50-95</sub><br>(pontuação global) | mAP<sub>50</sub> | Precisão<br>(↓ FP) | Recall<br>(↓ FN) |
|--------------------------------------|:-----------------------------------------:|:----------------:|:------------------:|:----------------:|
| **YOLOv8-s – baseline**              | **0,231** | **0,458** | **63,7 %** | 40,8 % |
| **YOLOv8-s – fine-tune (15 ép.)**    | 0,224 | 0,442 | 56,2 % | **44,6 %** |

#### Leitura dos números

* **Recall** passou de **40,8 % → 44,6 %** (+3,8 p.p.), ou seja, o sistema encontra ~1 em cada 25 lesões adicionais.  
* **Precisão** caiu de 63,7 % para **56,2 %** (-7,5 p.p.), implicando mais falsos-positivos que terão de ser revistos pelo radiologista.  
* A pontuação global **mAP 50-95** desceu ligeiramente (-0,007), um custo aceitável quando o objetivo é maximizar a segurança do doente.

#### Por que preferir maior recall?

Em contexto médico, **perder uma lesão (falso-negativo)** pode atrasar o diagnóstico e trazer consequências graves, enquanto um **falso-positivo** apenas acarreta uma verificação adicional pelo especialista. Portanto, aumentar a sensibilidade do sistema, mesmo à custa de alguns alarmes extra, alinha-se com as práticas de triagem radiológica.

#### Decisão final

> Adoptamos o **YOLOv8-s fine-tune (15 ép.)** como modelo final, priorizando a detecção de mais casos reais e minimizando o risco de falhas críticas.